In [ ]:
%reset

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from pathlib import Path
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, DistilBertModel
import pickle

def pkl_load(path):
  with open(path, 'rb') as file:
    return pickle.load(file)

In [ ]:
data_dir = Path('../input/commonlitreadabilityprize')
test_file = data_dir / 'test.csv'
sample_file = data_dir / 'sample_submission.csv'
submission_file = 'submission.csv'

In [ ]:
class SimpleModel(nn.Module):
  def forward(self, x):
    return [0.] * len(x)

model_simple = pkl_load("../input/simple-model/simple_model") # This one works

In [ ]:
class Model1(nn.Module):
  def __init__(self, tokenizer, model):
    super().__init__()
    self.tokenizer = tokenizer
    self.model = model
    self.model.to('cuda')

  def forward(self, x):
    toks = self.tokenizer(x, padding=True, return_tensors="pt")
    toks.to('cuda')
    return self.model(**toks).logits.flatten()

model_full = pkl_load("../input/model-out-3/model_out_3") # This one fails!

In [ ]:
def get_chunks(end, step):
    chunks = list(zip(range(0, end, step), range(step, end, step)))
    last_idx = 0 if len(chunks) == 0 else chunks[-1][1]
    if last_idx < end:
        chunks.append((last_idx, end))
    return chunks

In [ ]:
test_data = pd.read_csv(test_file).excerpt.tolist()
preds = []
for i_s, i_e in get_chunks(len(test_data), 16):
    preds += model_full(test_data[i_s:i_e]).tolist()
preds

In [ ]:
# test_data = pd.read_csv(test_file)
# preds = model_full(test_data.excerpt.tolist()).tolist()
# preds

In [ ]:
# NOTE: This works, if I uncomment it!
# sub = pd.read_csv(sample_file, index_col='id')
# sub.to_csv(submission_file)

In [ ]:
sub = pd.read_csv(sample_file, index_col='id')
sub.target = preds
sub.to_csv(submission_file)
sub